<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/train_omniversal_ai_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python3
"""
train_omniversal_ai.py

End-to-end pipeline for OmniversalAI:
1. Synthetic “omniversal” dataset of 6 inputs → 3 outputs
2. Standardization, noise injection
3. MLP with ReLU & optional Dropout (accepts int hidden_dims)
4. Physics-informed residual enforcing toy “omniverse” laws
5. MC-Dropout for uncertainty quantification
6. Training loop with AdamW, ReduceLROnPlateau, gradient clipping, NaN checks, early stopping
7. Safe checkpoint loading
8. Visualizations: loss curves, true vs. predicted scatter, uncertainty heatmap
"""

import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split

# ------------------------------------------------------------------------------
# 1. Synthetic Omniversal Dataset
# ------------------------------------------------------------------------------
class OmniversalDataset(Dataset):
    def __init__(self, n_samples=6000, seed=0):
        np.random.seed(seed)
        # Features: Reality coherence (RC), cross-dimensional entropy (CDE),
        # omniversal stability (OS), transdimensional flux (TF),
        # quantum synchrony (QS), causal coupling (CC)
        RC  = np.random.uniform(0.1, 1.0, (n_samples,1))
        CDE = np.random.uniform(0.0, 5.0, (n_samples,1))
        OS  = np.random.uniform(0.5, 2.0, (n_samples,1))
        TF  = np.random.uniform(0.0, 1.0, (n_samples,1))
        QS  = np.random.uniform(0.0, 1.0, (n_samples,1))
        CC  = np.random.uniform(0.1, 1.0, (n_samples,1))

        X_raw = np.hstack([RC, CDE, OS, TF, QS, CC]).astype(np.float64)

        # Toy “omniversal laws” targets:
        eps = 1e-6
        # 1. intelligence_alignment = RC * QS / (CDE + eps)
        IA = RC * QS / (CDE + eps)
        # 2. causality_integrity = OS * CC / (TF + eps)
        CI = OS * CC / (TF + eps)
        # 3. entropy_balance = CDE * CC * OS
        EB = CDE * CC * OS

        Y_raw = np.hstack([IA, CI, EB]).astype(np.float64)
        # 1% relative noise
        Y_raw += 0.01 * Y_raw.std(axis=0) * np.random.randn(*Y_raw.shape)

        # Compute normalization stats
        self.X_mean = X_raw.mean(axis=0)
        self.X_std  = X_raw.std(axis=0) + 1e-8
        self.Y_mean = Y_raw.mean(axis=0)
        self.Y_std  = Y_raw.std(axis=0) + 1e-8

        # Standardize
        self.X = ((X_raw - self.X_mean) / self.X_std).astype(np.float32)
        self.Y = ((Y_raw - self.Y_mean) / self.Y_std).astype(np.float32)

        # Debug range prints
        print(f"X range: {self.X.min():.3e}–{self.X.max():.3e}")
        print(f"Y range: {self.Y.min():.3e}–{self.Y.max():.3e}")

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx]), torch.from_numpy(self.Y[idx])

# ------------------------------------------------------------------------------
# 2. Model Definition (accept int hidden_dims)
# ------------------------------------------------------------------------------
class OmniversalAI(nn.Module):
    def __init__(self, input_dim=6, hidden_dims=(64,64), output_dim=3, p_drop=0.1):
        super().__init__()
        if isinstance(hidden_dims, int):
            hidden_dims = (hidden_dims,)
        layers, dim = [], input_dim
        for h in hidden_dims:
            layers += [
                nn.Linear(dim, h),
                nn.ReLU(),
                nn.Dropout(p_drop)
            ]
            dim = h
        layers.append(nn.Linear(dim, output_dim))
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

# ------------------------------------------------------------------------------
# 3. Physics-Informed Residual & Loss
# ------------------------------------------------------------------------------
def physics_residual(pred, X, stats):
    X_den = X * stats['X_std'] + stats['X_mean']
    RC, CDE, OS, TF, QS, CC = X_den.t()
    eps = 1e-6

    IA_t = RC * QS / (CDE + eps)
    CI_t = OS * CC / (TF + eps)
    EB_t = CDE * CC * OS
    Yt = torch.stack([IA_t, CI_t, EB_t], dim=1)

    Yt_norm = (Yt - stats['Y_mean']) / stats['Y_std']
    return nn.MSELoss()(pred, Yt_norm)

def total_loss(pred, true, X, stats, lam=1.0):
    mse  = nn.MSELoss()(pred, true)
    phys = physics_residual(pred, X, stats)
    return mse + lam * phys, mse, phys

# ------------------------------------------------------------------------------
# 4. MC-Dropout for Uncertainty Quantification
# ------------------------------------------------------------------------------
def mc_dropout_predict(model, X, T=50):
    model.train()
    samples = []
    with torch.no_grad():
        for _ in range(T):
            samples.append(model(X))
    stacked = torch.stack(samples, dim=0)
    return stacked.mean(0), stacked.std(0)

# ------------------------------------------------------------------------------
# 5. Training Loop with NaN Protection & Safe Checkpoint
# ------------------------------------------------------------------------------
def train(model, tr_loader, va_loader, stats, device,
          lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10):
    model.to(device)
    opt = optim.AdamW(model.parameters(), lr=lr, weight_decay=wd)
    sched = optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=5)

    best_val, wait = float('inf'), 0
    history = {'train': [], 'val': []}

    for ep in range(1, epochs+1):
        # Training
        model.train()
        run = 0.0
        for xb, yb in tr_loader:
            xb, yb = xb.to(device), yb.to(device)
            pred = model(xb)
            loss, mse, phys = total_loss(pred, yb, xb, stats, lam)

            if torch.isnan(loss):
                print(f"NaN loss at epoch {ep}, aborting.")
                return history

            opt.zero_grad()
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            run += loss.item() * xb.size(0)
        tr_loss = run / len(tr_loader.dataset)

        # Validation
        model.eval()
        run = 0.0
        with torch.no_grad():
            for xb, yb in va_loader:
                xb, yb = xb.to(device), yb.to(device)
                pred = model(xb)
                loss, _, _ = total_loss(pred, yb, xb, stats, lam)
                run += loss.item() * xb.size(0)
        va_loss = run / len(va_loader.dataset)

        sched.step(va_loss)
        history['train'].append(tr_loss)
        history['val'].append(va_loss)
        print(f"Epoch {ep:03d} ┃ Train {tr_loss:.4e} ┃ Val {va_loss:.4e}")

        # Checkpoint
        if va_loss < best_val - 1e-8:
            best_val, wait = va_loss, 0
            torch.save(model.state_dict(), "best_omniversal_ai.pth")
        else:
            wait += 1
            if wait >= patience:
                print("Early stopping.")
                break

    # Safe load
    if os.path.exists("best_omniversal_ai.pth"):
        model.load_state_dict(torch.load("best_omniversal_ai.pth", map_location=device))
    else:
        print("No best checkpoint found; using last epoch.")

    return history

# ------------------------------------------------------------------------------
# 6. Visualization Helpers
# ------------------------------------------------------------------------------
def plot_losses(hist):
    plt.figure()
    plt.plot(hist['train'], label='Train')
    plt.plot(hist['val'],   label='Val')
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.show()

def plot_scatter(y_true, y_pred, name):
    plt.figure()
    plt.scatter(y_true, y_pred, s=5, alpha=0.6)
    m, M = y_true.min(), y_true.max()
    plt.plot([m, M],[m, M], 'r--')
    plt.title(name); plt.xlabel("True"); plt.ylabel("Predicted"); plt.show()

def plot_uncertainty_heatmap(model, stats, device):
    # Vary CDE vs. OS; others at mean
    grid = 100
    CDE = np.linspace(0,5,grid, dtype=np.float32)
    OS  = np.linspace(0.5,2,grid, dtype=np.float32)
    G1, G2 = np.meshgrid(CDE, OS)
    pts = G1.size

    Xg = torch.zeros((pts,6), device=device)
    # RC, TF, QS, CC = means
    for i in (0,3,4,5):
        Xg[:,i] = stats['X_mean'][i]
    Xg[:,1] = torch.from_numpy(G1.ravel()).to(device)
    Xg[:,2] = torch.from_numpy(G2.ravel()).to(device)

    Xn = (Xg - stats['X_mean']) / stats['X_std']
    _, std = mc_dropout_predict(model, Xn, T=100)
    U = std[:,0].cpu().reshape(G1.shape)

    plt.figure(figsize=(5,4))
    plt.pcolormesh(G1, G2, U, cmap='magma', shading='auto')
    plt.colorbar(label='Std(Alignment)')
    plt.xlabel("Cross-Dimensional Entropy")
    plt.ylabel("Omniversal Stability")
    plt.title("Uncertainty Heatmap: Intelligence Alignment")
    plt.tight_layout()
    plt.show()

# ------------------------------------------------------------------------------
# 7. Main Execution
# ------------------------------------------------------------------------------
if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    ds = OmniversalDataset(n_samples=6000, seed=0)
    stats = {
        'X_mean': torch.tensor(ds.X_mean, dtype=torch.float32, device=device),
        'X_std' : torch.tensor(ds.X_std,  dtype=torch.float32, device=device),
        'Y_mean': torch.tensor(ds.Y_mean, dtype=torch.float32, device=device),
        'Y_std' : torch.tensor(ds.Y_std,  dtype=torch.float32, device=device),
    }

    n_val = int(0.2 * len(ds))
    tr_ds, va_ds = random_split(ds, [len(ds)-n_val, n_val])
    tr_ld = DataLoader(tr_ds, batch_size=128, shuffle=True)
    va_ld = DataLoader(va_ds, batch_size=256, shuffle=False)

    model = OmniversalAI(input_dim=6, hidden_dims=32, output_dim=3, p_drop=0.1).to(device)
    history = train(model, tr_ld, va_ld, stats, device,
                    lr=1e-4, wd=1e-5, lam=1.0, epochs=100, patience=10)

    plot_losses(history)

    # Scatter true vs. predicted
    X_all = torch.from_numpy(ds.X).to(device)
    with torch.no_grad():
        Yp_norm = model(X_all).cpu().numpy()
    Yp = Yp_norm * ds.Y_std + ds.Y_mean
    Yt = ds.Y * ds.Y_std + ds.Y_mean

    names = ["Intelligence Alignment", "Causality Integrity", "Entropy Balance"]
    for i, nm in enumerate(names):
        plot_scatter(Yt[:,i], Yp[:,i], nm)

    # Uncertainty heatmap
    plot_uncertainty_heatmap(model, stats, device)